
# TP 1 : Theorie des Graphes 1 
#### L3 INFO NEC 2024–2025 <br> Université de Pau et des Pays de l’Adour
###### date: "2024-12-18"

## prérequis

### installer avec pip (sur la machine)

c'est pareil avec Windows, Linux, Macos

##### version de python :

>si vous avez la version python ou python3 il suffit d'ajoute le 3 en fonction de votre version<br>
>et heuresement python fonctionne avec pip et python3 fonctionne avec pip3 (normalement les versions<br>
>actuelles installées sont pip3 avec python3, python n'est plus utilisé donc pip non plus)<br>

#### installer dash_cytoscape avec 2 packages
```sh
pip3 install dash
pip3 install dash-cytoscape
```

#### installer pip3 :

```sh
python3 -m ensurepip --upgrade
```
enlever 3 dans python3 pour les version ultérieures<br>
si problèmes voir : https://pip.pypa.io/en/stable/installation/



### importation dans le code

In [46]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan


En premier, il faut définir les variables d'entrée: le graphe. <br>
Ce graphe est donné par un vecteur des sommets, une matrice des arêtes, et un vecteur des poids. <br>

Le graphe avec lequel on va tester notre code est simple: <br>
On va avoir 6 sommets et ça c'est la liste de sommets adjacentes avec les poids : 

### on crée nos classes pour créer une structure de graphe

In [16]:
class Vertex:
    """gère les sommets et ses arrêtes (sous forme d'autres sommets)"""

    id = 0 # identifiant unique de tous les vertex dans les graphes
    
    def __init__(self, name, weight=0):
        """name:le nom du sommet (vertex) associé au noeud
        weight: le poids du sommet pour aller du noeud jusq'au sommet (par défaut=0)
        (nom du noeud non représenté ici)"""
        self.name = name
        self.weight = weight # du sommet du noeud jusq'a ce sommet (ici)
        self.id = Vertex.id
        Vertex.id += 1
    
    def __str__(self):
        '''si on fait print(Vertex) le resultat du print est dans ce return'''
        return self.name

In [17]:
class Node:
    """
    représente les noeuds entre des point dans un graphe
    
    on crée la classe qui s'occupe de tous les noeuds entre 
    les sommets (vertex/vertecies) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant unique de noeud auquel il appartient    

    def __init__(self, node_vertex:Vertex, vertecies:list[Vertex]):
        self.name = node_vertex.name
        self.weight = node_vertex.weight # si jamais on peut revenir vers le sommet lui même (boucle)
        self.vertecies = vertecies
        self.degree = len(vertecies)
        self.id = Node.id
        Node.id += 1

    def __str__(self, print_weight:bool=True):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.vertecies)
        for i, e in enumerate(self.vertecies):
            res += f"{e.name}"
            if(print_weight):
                res += f" w={e.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res += "," 
                res+=" "
        res+="]"
        return res
    
    def _sort(self, vertecies_:list[Vertex])->list[Vertex]: # complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances"""
        if len(vertecies_)<2 :
            return vertecies_
        else:
            # elem du milieu
            pivot = vertecies_[len(vertecies_)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for v in vertecies_:
                if(v.weight < pivot): l.append(v)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(v.weight == pivot): m.append(v) 
                else: r.append(v)
            return self._sort(l)+m+self._sort(r)
        
    def sort_by_weight(self):
        '''trie la liste des sommets (vetecies) par poids croissant (ASC)'''
        # on utilise une fonction réccursive donc code separement
        self.vertecies = self._sort(self.vertecies)
        # futures versions : trier de manière reccursive sans faire de
        # seconde fonction (_sort(...)) qui stock dans 3 tableaux temporaire
        # la duplication des valeurs d'origine (pour + de performances)




In [43]:

class Graph:

    def __init__(self, data:list[Node], title:str="default"):
        self.title = title # titre du graphique si utilisé
        self.adj = data # matrice d'adjacence
        self.matrix = self.create_matrix() # matrice des liens entre les noeuds (Edges)
        self.weight = [[nod.name for nod in node.vertecies] for node in data] # poids des trajets
        self.nodes = [node.name for node in data] # liste des noeuds (seulement les noms)
        self.degree = 0 # pas encore calculé le degré du graphe

    def create_matrix(self):
        """créer la matrice d'ajdacence des points
        (fonctionne avec les graphes orienté également)
        """
        self.matrix=[]
        for line, node in enumerate(self.adj):
            self.matrix.append([])
            for vertex in node.vertecies:
                self.matrix[line].append(
                    node.name == vertex.name
                )


    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affiche le plus simplement avec des caractères le graph"""
        res = ""
        for i, node in enumerate(self.adj):
            res += f"{i}| {node.name} ["
            v_size = len(node.vertecies)
            for j in range(v_size):
                res += f"{node.vertecies[j].name}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        """
        for node in self.adj:
            node.vertecies.sort_by_weight()
    
    def render(self):
        """effectue le rendu du graphe visuellement"""
        ...

In [44]:
# test
nodes = Node(Vertex("A"), [
        Vertex("B", 2), 
        Vertex("C", 1), 
        Vertex("D", 3), 
        Vertex("E", 7), 
        Vertex("F", 5)
    ]
)
print(nodes)
nodes.sort_by_weight()
print(nodes)


A [B w=2, C w=1, D w=3, E w=7, F w=5]
A [C w=1, B w=2, D w=3, F w=5, E w=7]


## saisie des données

#### données fournies :

> 1: 2(2), 3(1)<br>
> 2: 1(2), 4(2), 5(3)<br>
> 3: 1(1), 2(3), 4(2)<br>
> 4: 2(2), 3(5), 5(2), 6(4)<br>
> 5: 2(3), 4(2), 6(2)<br>
> 6: 4(2), 5(2)<br>


In [45]:
# rappel format du noeud : 
# Noeud(nomActuel, [Vertex("nomLié1", poid1),Vertex("nomLié2", poid2),etc...])
data = [
    Node(Vertex("1"),[Vertex("2",2),Vertex("3",1)]),
    Node(Vertex("2"),[Vertex("1",2),Vertex("4",2),Vertex("5",3)]),
    Node(Vertex("3"),[Vertex("1",1),Vertex("2",3),Vertex("4",2)]),
    Node(Vertex("4"),[Vertex("2",2),Vertex("3",5),Vertex("5",2),Vertex("6",4)]),
    Node(Vertex("5"),[Vertex("2",3),Vertex("4",2),Vertex("6",2)]),
    Node(Vertex("6"),[Vertex("4",2),Vertex("5",2)])
]
graphe = Graph(data,"exemple de graphe")
print(graphe.matrix)

None


## Kruskal

> init: arrêtes d'ordre ascendant de poids

on peut utiliser `sort()` qui existe a la fois dans python et dans RStudio<br>
> pour i=1...n-1 des sommets<br>
> prendre l'arrête de poids min qui ne fait pas une boucle<br>
> fin

In [25]:
# on fait de l'héritage car on peut pas 
# rajouter la méthode kruskal comme en swift avec des extensions
class GraphK(Graph):

    # obligatoire (heritage)
    def __init__(self, data, title = "default"):
        super().__init__(data, title)

    # obligatoire
    def __init__(self, gr:Graph):
        """convertit un Graph normal en GraphK"""
        super().__init__(gr.adj,gr.title)
    
    def kruskal(self,red_tarjan_rule=False)->list[any]:
        """renvoie l'arbre couvrant de poids minimal
        applique la ègle rouge de tarjan si booléen est True (False par défaut)"""
        # comme vue en cours
        self.sort_by_weight() # trie tout le graphe
        tmin = []
        carry = True # stopper la boucle
        idx = 0
        for node in self.adj: # pour chaque sommets (du noeud)
            # 1er arrete qui n'est pas dans tmin
            while(carry and idx < len(tmin)): # parcours de tmin
                if(tmin[idx] not in [node.name+v for v in node.vertecies]):
                    pass
                idx+=1
            # reset
            carry=True
            idx = 0

            a = node.vertecies[0] 


        

In [41]:
# vérification (données insérées a la main pour être sûr)
graphe = GraphK(graphe) # conversion
print(graphe)
#graphe.sort_weight_ASC()
#print(graphe)

0| 1 [2, 3]
1| 2 [1, 4, 5]
2| 3 [1, 2, 4]
3| 4 [2, 3, 5, 6]
4| 5 [2, 4, 6]
5| 6 [4, 5]



methode kruskall en R : 
```R
kruskal <- function(sommets,arretes,poids){
    poids_ord <- sort(poids,index.return=true)
    poids <- poids_ord$x
    index_poids <- poids_ord$ix 
    return poids
}
```